### Fine tune LLM embeedings model


In [2]:
#!pip install openai==0.28.1
#!pip install openai --upgrade
#!pip install ragas
#!pip install unstructured
#!pip install langchain[all]
#!pip install --upgrade langchain

#!pip install playwright
#!pip install -U selenium unstructured
#!pip install --upgrade langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

In [1]:
#!pip install pydantic==2.5
#!pip install --upgrade --quiet  langchain_milvus
#!pip install --upgrade scipy

In [1]:
#!pip install -U langchain-openai

In [2]:
import os, json
#import openai
#from langchain.chat_models import ChatOpenAI, ChatGooglePalm
from utils import OPENAI_API_KEY


os.environ['OPENAI_API_KEY'] =  OPENAI_API_KEY
from llm_utils import *
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

#openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
from transformers import (BertTokenizer, 
                          BertModel,
                          BertForMaskedLM, 
                          Trainer, 
                          TrainingArguments, 
                          DataCollatorForLanguageModeling,
                          DataCollatorForWholeWordMask,
                          DataCollatorWithPadding
                         )
from datasets import load_dataset


In [5]:
#from langchain.chat_models import ChatOpenAI
import langchain_core
from langchain_openai import ChatOpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain_community.document_loaders import SeleniumURLLoader, TextLoader
from langchain_text_splitters import (RecursiveCharacterTextSplitter, 
                                      TextSplitter,
                                      NLTKTextSplitter,
                                      SentenceTransformersTokenTextSplitter
                                     )
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

ai_model = "gpt-4o-mini"

llm = ChatOpenAI(model=ai_model, temperature=0.)

### Load Documents

In [6]:
def load_url_documets(list_urls, 
                      spliter="nltk", 
                      tokens=512, 
                      overlap=0.1,
                      embeedding_model=OpenAIEmbeddings(),
                     ):
    
    overlap=int(overlap*tokens) if overlap>0 else 0
    # Load, chunk and index the contents of the blog.
    loader_url =SeleniumURLLoader( list_urls)
    docs = loader_url.load()
    if  spliter == "nltk":
        text_splitter = NLTKTextSplitter(chunk_size=tokens, 
                                         chunk_overlap=overlap)
        
    elif spliter == "sentence":
        text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=tokens,
                                                              model_name ="BAAI/bge-base-en-v1.5",
                                                              chunk_overlap=overlap)
        
    elif spliter == "recursive":
        text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n","\n"],
                                                       chunk_size=tokens, 
                                                       chunk_overlap=overlap)
        
    elif spliter == "semantic":
        text_splitter = SemanticChunker(embeedding_model,
                                       breakpoint_threshold_amount=None,
                                       number_of_chunks=None) 
                                                       
                                                       
    else:
        print( f"Unknown spliter {spliter}")
    
    #text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n","\n"], chunk_size=500, chunk_overlap=50)
    #text_splitter = NLTKTextSplitter(chunk_size=tokens, chunk_overlap=int(overlap*tokens))
    #text_splitter = SentenceTransformersTokenTextSplitter()
    splits = text_splitter.split_documents(docs)
    
    return splits, docs

In [4]:
url_list = [
           "https://www.nature.com/articles/s41524-023-01062-z",
        #    "https://www.nature.com/articles/s41699-023-00369-1",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/tree/main/docs/CONSTRUCTOR-MOCK.md"
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/CONSTRUCTOR.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/DATA.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/ENVIRONMENT.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-CONSTRUCTOR.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/GENERATING-MOCK.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/PILOT.md",
        #    "https://github.com/HSE-LAMBDA/ai4material_design/blob/main/docs/SPARSE-PAPER.md"
        #    "https://www.nature.com/articles/s41377-024-01407-3",
        #    "https://www.nature.com/articles/s41565-023-01407-1",
        #    "https://www.nature.com/articles/s41699-023-00369-1",
           ]
                               
chunks, documents = load_url_documets(url_list, spliter ="nltk", tokens=50)
print(len(chunks), len(documents))

Created a chunk of size 1063, which is longer than the specified 50
Created a chunk of size 137, which is longer than the specified 50
Created a chunk of size 110, which is longer than the specified 50
Created a chunk of size 146, which is longer than the specified 50
Created a chunk of size 143, which is longer than the specified 50
Created a chunk of size 128, which is longer than the specified 50
Created a chunk of size 129, which is longer than the specified 50
Created a chunk of size 539, which is longer than the specified 50
Created a chunk of size 171, which is longer than the specified 50
Created a chunk of size 210, which is longer than the specified 50
Created a chunk of size 177, which is longer than the specified 50
Created a chunk of size 253, which is longer than the specified 50
Created a chunk of size 123, which is longer than the specified 50
Created a chunk of size 87, which is longer than the specified 50
Created a chunk of size 227, which is longer than the specifie

Created a chunk of size 310, which is longer than the specified 50
Created a chunk of size 146, which is longer than the specified 50
Created a chunk of size 112, which is longer than the specified 50
Created a chunk of size 117, which is longer than the specified 50
Created a chunk of size 147, which is longer than the specified 50
Created a chunk of size 105, which is longer than the specified 50
Created a chunk of size 104, which is longer than the specified 50
Created a chunk of size 74, which is longer than the specified 50
Created a chunk of size 265, which is longer than the specified 50
Created a chunk of size 87, which is longer than the specified 50
Created a chunk of size 174, which is longer than the specified 50
Created a chunk of size 73, which is longer than the specified 50
Created a chunk of size 96, which is longer than the specified 50
Created a chunk of size 458, which is longer than the specified 50
Created a chunk of size 200, which is longer than the specified 50

Created a chunk of size 54, which is longer than the specified 50
Created a chunk of size 139, which is longer than the specified 50
Created a chunk of size 109, which is longer than the specified 50
Created a chunk of size 194, which is longer than the specified 50
Created a chunk of size 52, which is longer than the specified 50
Created a chunk of size 64, which is longer than the specified 50
Created a chunk of size 130, which is longer than the specified 50
Created a chunk of size 161, which is longer than the specified 50
Created a chunk of size 90, which is longer than the specified 50
Created a chunk of size 59, which is longer than the specified 50
Created a chunk of size 95, which is longer than the specified 50
Created a chunk of size 84, which is longer than the specified 50
Created a chunk of size 102, which is longer than the specified 50
Created a chunk of size 71, which is longer than the specified 50
Created a chunk of size 65, which is longer than the specified 50
Crea

399 1


In [6]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain

ai_model = "gpt-4o-mini"

llm = ChatOpenAI(model=ai_model, temperature=0.)

qa_chain = load_qa_chain(llm, chain_type="map_reduce")

qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain, 
                                         text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, 
                                                                                      chunk_overlap=100))

answer = qa_document_chain.run(
    input_document=documents[0].page_content,
    question=("""
     You are an expert in analyzing scientific papers.
     Read the text carefully and answer the question:
     What are the main three scientific subject areas of research and text study?
     Give the answer as a list.
    """
            )
             )
print(answer)

/home/oleg/miniconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


1. Machine Learning
2. Materials Science
3. Defect Analysis in Crystals


In [7]:
print(" ".join([ch.page_content for ch in chunks[1:5]]))
print()
print(chunks[5].page_content)

One of the most exciting characteristics of 2D crystals is the ability to tune their properties via controllable introduction of defects. However, the search space for such structures is enormous, and ab-initio computations prohibitively expensive. We propose a machine learning approach for rapid estimation of the properties of 2D material given the lattice structure and defect configuration. The method suggests a way to represent configuration of 2D materials with defects that allows a neural network to train quickly and accurately.

We compare our methodology with the state-of-the-art approaches and demonstrate at least 3.7 times energy prediction error drop.


### Data preparation

In [11]:
import langchain_core
def prepare_example_conversation(sentence1, sentence2, expert):
    """
    Create the data of QA the model.
    """
    messages = []
    
    if type(sentence1) == list:
        content1 = " ".join([s.page_content for s in sentence1])
    elif type(sentence1) == str:
        content1 = sentence1
    elif type(sentence1) == langchain_core.documents.base.Document:
        content1 = sentence1.page_content
    else:
        print("Unknown type of seentence1")
        
    if type(sentence2) == list:
        content2 = " ".join([s.page_content for s in sentence2])
    elif type(sentence2) == str:
        content2 = sentence2
    elif type(sentence2) == langchain_core.documents.base.Document:
        content2 = sentence2.page_content
    else:
        print("Unknown type of seentence2")
        
    system_message = f"You are an expert  in {expert}. Your task is to continue given user's text."
    
    messages = []
    messages.append({"role": "system", "content": system_message})

    messages.append({"role": "user", "content": content1})

    messages.append({"role": "assistant", "content": content2})

    return {"messages": messages} 

    
#{"prompt": "The quick brown fox jumps over the lazy dog.\n\n", "completion": "The fox then ran towards the forest, ready for the next adventure."}
#{"prompt": "It was a sunny day and the birds were chirping.\n\n", "completion": "The park was filled with families enjoying the weather and children playing."}



#print(prepare_example_conversation(chunks[0:5], chunks[5]))

In [12]:
expert = "  ".join([w.strip() for w in ["Machine Learning, Materials Science, Quantum Phyics,"]
                     ])
expert


'Machine Learning, Materials Science, Quantum Phyics,'

In [13]:
train_size = 1
win_size = 5
tr_int = int(len(chunks) * train_size)

training_dataset = []

                
for i  in range(1, len(chunks) - win_size):
    
    training_dataset.append(prepare_example_conversation(chunks[i:i+win_size], chunks[i+win_size], expert))
    
print(len(training_dataset))       

393


In [14]:
def clear_prompts(data):
    #c_data = []
    for d in data:
        c_data = []
        for  m in d["messages"]:
            #m["content"] = m["content"].replace("\n\n", "\n")
            c_data.append( {"role": m["role"], "content": m["content"].replace( "\n", " ")})
        d["messages"] =  c_data

In [15]:
def clear_prompts(data):
    c_data = []
    for d in data:
        _data = []
        for  m in d["messages"]:
            if len(m["content"].split("\n")) == 1:
                _data.append( {"role": m["role"], "content": m["content"]})
        if len(_data) == 3:
            c_data.append({"messages":  _data})
    return c_data

In [16]:
cl_data = clear_prompts(training_dataset)
print(len(cl_data))

104


In [17]:
train_size = 0.8

training_data = cl_data[:int(len(cl_data) * train_size)]
validation_data = cl_data[int(len(cl_data) * train_size):]
print(len(training_data), len(validation_data))

83 21


In [18]:
training_data[1]

{'messages': [{'role': 'system',
   'content': "You are an expert  in Machine Learning, Materials Science, Quantum Phyics,. Your task is to continue given user's text."},
  {'role': 'user',
   'content': 'Controllable defect engineering, i.e., introduction of vacancies or desired impurities, enables properties modifications and new functionalities in crystalline materials1. The opportunities for such controlled material engineering methods got a dramatic boost in the past two decades with the development of the methods for exfoliation of crystal into two-dimensional atomic layer2. The reduced dimensionality in layered two-dimensional materials makes it possible to manipulate defects atom by atom and tune their properties down to quantum mechanics limits3. Such atomic-scale preparation and fabrication techniques hold promise for the continual development of the semiconductor industry in the post-Moore age and the development of novel technologies such as quantum computing4, catalysts5, 

In [24]:
def write_jsonl(data_list: list, filename: str) -> None:
    """
    Write a training file
    """
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [17]:
training_file_name = "tmp_ai4material_training_cl.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_ai4material__validation_cl.jsonl"
write_jsonl(validation_data, validation_file_name)

### Upload files

In [190]:
import openai

client = openai.OpenAI(api_key=OPENAI_API_KEY )

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id


validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-NYumpxMVH0tOAc5DunIk2qbp
Validation file ID: file-szEUv3gDWdfwLrToddu5XIwH


### Fine tuning

In [191]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-4o-mini-2024-07-18",  # gpt-3.5-turbo-1106
    suffix="ai4materials",
    hyperparameters = {"batch_size": 4,
                       "learning_rate_multiplier": 1e-4,
                       "n_epochs": 3}
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-klyexzicQmFqofpDUS4350Ur
Status: validating_files


In [192]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-klyexzicQmFqofpDUS4350Ur
Status: validating_files
Trained Tokens: None


In [193]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()


for event in events:
    print(event.message)

Created fine-tuning job: ftjob-klyexzicQmFqofpDUS4350Ur
Validating training file: file-NYumpxMVH0tOAc5DunIk2qbp and validation file: file-szEUv3gDWdfwLrToddu5XIwH


### Masking Fine-tuning

In [19]:
import numpy as np

In [20]:
def  mask_text(text, p=0.3, ns=4):
    sl = text.split(' ')
    sl_len = list(map(len, sl))
    #mask = list(map(lambda x: random.random() <1-p if x else True > ns,  sl_len))
    #return " ".join([s if m else " ... " for s,m in zip(sl, mask) ])
    
    mask = np.random.choice([True, False], size=len(sl), p=[1 -p, p])
    return " ".join([s if (m or n < ns) else " ... " for s,m,n in zip(sl, mask, sl_len) ])

s = training_data[1]["messages"][1]["content"]
print(s)
mask_text(s, 0.1, 4)

Controllable defect engineering, i.e., introduction of vacancies or desired impurities, enables properties modifications and new functionalities in crystalline materials1. The opportunities for such controlled material engineering methods got a dramatic boost in the past two decades with the development of the methods for exfoliation of crystal into two-dimensional atomic layer2. The reduced dimensionality in layered two-dimensional materials makes it possible to manipulate defects atom by atom and tune their properties down to quantum mechanics limits3. Such atomic-scale preparation and fabrication techniques hold promise for the continual development of the semiconductor industry in the post-Moore age and the development of novel technologies such as quantum computing4, catalysts5, and photovoltaics6. Despite decades of research efforts, knowledge of the structure-property relation for defects in crystals is still limited.


'Controllable defect engineering, i.e., introduction of vacancies or desired impurities, enables properties modifications and new functionalities in crystalline materials1. The opportunities for such controlled material  ...  methods got a dramatic  ...  in the past two decades with the development of the methods for exfoliation of crystal into two-dimensional atomic layer2. The reduced  ...  in layered two-dimensional materials makes it possible to  ...  defects atom by atom and tune their properties down to quantum mechanics limits3. Such atomic-scale  ...  and  ...  techniques hold promise for the continual development of the  ...  industry in the post-Moore age and the development of novel technologies such as quantum computing4, catalysts5, and photovoltaics6. Despite decades of research  ...  knowledge of the structure-property relation for defects in crystals is still limited.'

In [21]:
def masking_data(data, expert, p=0.2):
    
    system_message = f"""You are an expert in {expert}. 
    Your task is to fill in the gaps marked ... in this text with the most appropriate words 
    based on the context and area of expertise. """
    
    masked_data = []
        
    for d in data:
        masked_messages = []
        
        i = 0
        m = d["messages"][i]
        while m['role'] != 'user':
            m = d["messages"][i]
            content = m["content"]
            i += 1
            
        masked_messages.append({"role": "system", "content": system_message})
        masked_messages.append({"role": "user", "content": mask_text(content, p)})
        masked_messages.append({"role": "assistant", "content": content})
        
        masked_data.append({"messages": masked_messages})
    return masked_data



In [21]:
masked_training_data = masking_data(training_data, expert, 0.1)
masked_validation_data = masking_data(validation_data, expert, 0.1)

In [398]:
masked_training_data[0]

{'messages': [{'role': 'system',
   'content': 'You are an expert in Machine Learning, Materials Science, Quantum Phyics,. \n    Your task is to fill in the gaps marked ... in this text with the most appropriate words \n    based on the context and area of expertise. '},
  {'role': 'user',
   'content': 'One of the  ...  exciting characteristics of 2D crystals is the ability to tune their properties via controllable introduction of defects. However, the  ...  space for such structures is enormous, and ab-initio  ...  prohibitively expensive. We propose a machine learning approach for rapid estimation of the properties of 2D material given the lattice structure and defect configuration. The method suggests a way to represent configuration of 2D materials with defects that allows a neural network to train quickly and accurately. We compare our methodology with the  ...  approaches and demonstrate at least 3.7 times energy prediction error drop.'},
  {'role': 'assistant',
   'content': 'O

In [399]:
training_file_name = "tmp_ai4material_training_cl.jsonl"
write_jsonl(masked_training_data, training_file_name)

validation_file_name = "tmp_ai4material__validation_cl.jsonl"
write_jsonl(masked_validation_data, validation_file_name)

### Upload files

In [274]:
import openai

client = openai.OpenAI(api_key=OPENAI_API_KEY )

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id


validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-j61cZitIP6VwyJzEhVwQw9ks
Validation file ID: file-gaxBl02EVDNNXMLcIRVs53cC


### Fine tuning

In [275]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-4o-mini-2024-07-18",  # gpt-3.5-turbo-1106
    suffix="ai4materials_v0",
    hyperparameters = {"batch_size": 4,
                       "learning_rate_multiplier": 1e-4,
                       "n_epochs": 3}
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-AOO3sWe83ikrlZVo4wc4OLVX
Status: validating_files


In [276]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-AOO3sWe83ikrlZVo4wc4OLVX
Status: validating_files
Trained Tokens: None


In [277]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()


for event in events:
    print(event.message)

Created fine-tuning job: ftjob-AOO3sWe83ikrlZVo4wc4OLVX
Validating training file: file-j61cZitIP6VwyJzEhVwQw9ks and validation file: file-gaxBl02EVDNNXMLcIRVs53cC


In [22]:
def fineTuningOpenai(data_tr, 
                     data_val,
                     expert, 
                     file_name,
                     p=0.1,
                     model="gpt-4o-mini-2024-07-18",  # gpt-3.5-turbo-1106
                     suffix="ai4materials_v0",
                     batch_size=4,
                     learing_rate_multiplier=1e-4,
                     n_epochs=3
                    ):
                     
    import openai
    
    masked_training_data = masking_data(data_tr, expert, p)
    masked_validation_data = masking_data(data_val, expert, p)
    print("Masked training data", len(masked_training_data), "validation data", len(masked_validation_data))
    data = { "training": masked_training_data, 
            "validation": masked_validation_data}
    
    training_file_name = f"data/data_rag/{file_name}_training.jsonl"
    write_jsonl(masked_training_data, training_file_name)

    validation_file_name = f"data/data_rag/{file_name}_validation.jsonl"
    write_jsonl(masked_validation_data, validation_file_name)
    
    client = openai.OpenAI(api_key=OPENAI_API_KEY )

    training_response = client.files.create(
        file=open(training_file_name, "rb"), purpose="fine-tune"
    )
    training_file_id = training_response.id


    validation_response = client.files.create(
        file=open(validation_file_name, "rb"), purpose="fine-tune"
    )
    validation_file_id = validation_response.id

    print("Training file ID:", training_file_id)
    print("Validation file ID:", validation_file_id)
    
    response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=model,  # gpt-4o-mini-2024-07-18
    suffix=suffix, #ai4materials_v0
    hyperparameters = {"batch_size": batch_size,
                       "learning_rate_multiplier": learing_rate_multiplier,
                       "n_epochs": n_epochs}
    )

    job_id = response.id

    print("Job ID:", response.id)
    print("Status:", response.status)
    
    response = client.fine_tuning.jobs.retrieve(response.id)

    print("Job ID:", response.id)
    print("Status:", response.status)
    print("Trained Tokens:", response.trained_tokens)

    return response, job_id, data


In [31]:
p =0.2
response, job_id, data = fineTuningOpenai(training_data,
                 validation_data,
                 expert,
                 f"tmp_ai4material_v{p}",
                 p=p,
                 #model="gpt-4o-mini-2024-07-18",  # gpt-3.5-turbo-1106
                 model = "text-embedding-ada-002",
                 suffix=f"ai4materials_v{p}",
                 batch_size=8,
                 learing_rate_multiplier=1e-6,
                 n_epochs=3)




Masked training data 83 validation data 21
Training file ID: file-q7y2Vx82TCeXLE76uFf1wKqt
Validation file ID: file-e8nYglfyXWj2vmftK793KFxt


BadRequestError: Error code: 400 - {'error': {'message': 'Model text-embedding-ada-002 is not available for fine-tuning or does not exist.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}

In [28]:
import openai
client = openai.OpenAI(api_key=OPENAI_API_KEY )
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()


for event in events:
    print(event.message)

Created fine-tuning job: ftjob-fX6xeJf2NPGF8ypU7YafOKgR
Validating training file: file-ivEcZtskI3Me4DchryNXsJ4C and validation file: file-YOIaUvi37c5eFr4a7wj5UyiM
Files validated, moving job to queued state
Fine-tuning job started
Step 1/32: training loss=0.48, validation loss=0.40
Step 2/32: training loss=0.60, validation loss=0.23
Step 3/32: training loss=0.56, validation loss=0.46
Step 4/32: training loss=0.52, validation loss=0.26
Step 5/32: training loss=0.63, validation loss=0.27
Step 6/32: training loss=0.62, validation loss=0.38
Step 7/32: training loss=0.44, validation loss=0.32
Step 8/32: training loss=0.48, validation loss=0.30
Step 9/32: training loss=0.62, validation loss=0.34
Step 10/32: training loss=0.57, validation loss=0.22, full validation loss=0.27
Step 11/32: training loss=0.34, validation loss=0.42
Step 12/32: training loss=0.60, validation loss=0.37
Step 13/32: training loss=0.63, validation loss=0.21
Step 14/32: training loss=0.41, validation loss=0.42
Step 15/3

In [413]:
event.object

'fine_tuning.job.event'

### Inference

In [23]:
def get_completion(messages, model):
    
    try:
        from openai import ChatCompletion
        
        response = ChatCompletion.create(
                   model=model,
                   messages=messages,
                   temperature=0, 
                    )
        
        return response#.choices[0].message["content"]
    
    except:
        from openai import OpenAI
        
        client = OpenAI(api_key=OPENAI_API_KEY )
        response = client.chat.completions.create(
                          model=model,
                   messages=messages,
                   temperature=0 
                   )
        return response.choices[0].message.content  

In [29]:
data["validation"][0]

{'messages': [{'role': 'system',
   'content': 'You are an expert in Machine Learning, Materials Science, Quantum Phyics,. \n    Your task is to fill in the gaps marked ... in this text with the most appropriate words \n    based on the context and area of expertise. '},
  {'role': 'user',
   'content': 'We use a value averaged over 12 experiments, same as in  ...  2 to estimate training stability. For formation  ...   ...   ...  the z coordinate difference in sparse representation edges  ...  the Sparse-Z model to  ...  the Full model everywhere except h-BN; adding pristine atom species  ...  as the node features contributes the most of the remaining gain. The most likely explanation for the importance of the pristine  ...  for h-BN is  ...  both atoms can be substituted to C, without this additional information, the model can’t distinguish between B and N substitutions. Adding EOS improves expected prediction quality and stability by a small amount for the low-density datasets. For H

In [405]:
print(data["validation"][0]['messages'][1]['content'])
print()
print(data["validation"][0]['messages'][2]['content'])

We use a value averaged over 12 experiments, same as in Table 2 to estimate training stability. For formation  ...  just enabling the z coordinate difference in sparse representation edges allows the Sparse-Z model to outperform the Full model  ...  except h-BN; adding pristine  ...  species (Sparse-Z-Were) as the node  ...  contributes the most of the remaining gain. The most likely explanation for the importance of the pristine species for  ...  is that both atoms can be substituted to C, without this additional information, the model can’t distinguish between B and N substitutions. Adding EOS  ...  expected prediction quality and stability by a small amount for the low-density datasets. For  ...  gap, Sparse-Z-Were and Sparse-Z-Were-EOS perform similarly to  ...  in terms of the combined metric, outperform it by a factor of 4 for low-density data.

We use a value averaged over 12 experiments, same as in Table 2 to estimate training stability. For formation energy, just enabling the 

In [410]:
j=0
test_messages = []
test_messages.append({"role": "system", 
                      "content": data["validation"][j]['messages'][0]['content']})

test_messages.append({"role": "user", 
                      "content": data["validation"][j]['messages'][1]['content']})
print(test_messages[1]['content'])
print()
print(data["validation"][j]['messages'][2]['content'])

get_completion(test_messages, 
               "ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v01:9xuixRUZ")


We use a value averaged over 12 experiments, same as in Table 2 to estimate training stability. For formation  ...  just enabling the z coordinate difference in sparse representation edges allows the Sparse-Z model to outperform the Full model  ...  except h-BN; adding pristine  ...  species (Sparse-Z-Were) as the node  ...  contributes the most of the remaining gain. The most likely explanation for the importance of the pristine species for  ...  is that both atoms can be substituted to C, without this additional information, the model can’t distinguish between B and N substitutions. Adding EOS  ...  expected prediction quality and stability by a small amount for the low-density datasets. For  ...  gap, Sparse-Z-Were and Sparse-Z-Were-EOS perform similarly to  ...  in terms of the combined metric, outperform it by a factor of 4 for low-density data.

We use a value averaged over 12 experiments, same as in Table 2 to estimate training stability. For formation energy, just enabling the 

'We use a value averaged over 12 experiments, same as in Table 2 to estimate training stability. For formation **energy**, just enabling the z coordinate difference in sparse representation edges allows the Sparse-Z model to outperform the Full model **in all cases** except h-BN; adding pristine **carbon** species (Sparse-Z-Were) as the node **feature** contributes the most of the remaining gain. The most likely explanation for the importance of the pristine species for **model performance** is that both atoms can be substituted to C, without this additional information, the model can’t distinguish between B and N substitutions. Adding EOS **(Equation of State)** expected prediction quality and stability by a small amount for the low-density datasets. For **the band** gap, Sparse-Z-Were and Sparse-Z-Were-EOS perform similarly to **the baseline model** in terms of the combined metric, outperform it by a factor of 4 for low-density data.'

### Next step MASKING p =0.2

In [32]:
p =0.2
response, job_id, data = fineTuningOpenai(training_data,
                 validation_data,
                 expert,
                 f"tmp_ai4material_v{p}",
                 p=p,
                 model="ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v01:9xuixRUZ",  # gpt-3.5-turbo-1106
                 suffix=f"ai4materials_v{p}",
                 batch_size=4,
                 learing_rate_multiplier=1e-6,
                 n_epochs=2,
                                         )




Masked training data 83 validation data 21
Training file ID: file-6C51nKKQyegyG0pJakAjAeyD
Validation file ID: file-ixGnmLjjc16GB7HBR5rVy9UV
Job ID: ftjob-Gf93iwgLnje3rlsI7Z5rmL9Z
Status: validating_files
Job ID: ftjob-Gf93iwgLnje3rlsI7Z5rmL9Z
Status: validating_files
Trained Tokens: None


In [33]:
import openai
client = openai.OpenAI(api_key=OPENAI_API_KEY )
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()


for event in events:
    print(event.message)

Created fine-tuning job: ftjob-Gf93iwgLnje3rlsI7Z5rmL9Z
Validating training file: file-6C51nKKQyegyG0pJakAjAeyD and validation file: file-ixGnmLjjc16GB7HBR5rVy9UV


In [419]:
j=1
test_messages = []
test_messages.append({"role": "system", 
                      "content": data["validation"][j]['messages'][0]['content']})

test_messages.append({"role": "user", 
                      "content": data["validation"][j]['messages'][1]['content']})
print(test_messages[1]['content'])
print()
print(data["validation"][j]['messages'][2]['content'])

get_completion(test_messages, 
               "ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v0-2:9xuy5Jqv")


Their  ...  nature makes them prone to  ...  modification,  ...  further increases their  ...  for a variety of applications. However, the search space for possible configurations is  ...  Thus, the ability to predict the  ...  of such crystals efficiently  ...  a vital task. In this  ...  we focus on predicting the properties of  ...  crystals blended with defects, substitutions, and  ...  State-of-the-art  ...  learning algorithms struggle to learn the properties of crystals’ defects accurately.

Their two-dimensional nature makes them prone to chemical modification, which further increases their tunability for a variety of applications. However, the search space for possible configurations is vast. Thus, the ability to predict the properties of such crystals efficiently becomes a vital task. In this paper, we focus on predicting the properties of such crystals blended with defects, substitutions, and vacancies. State-of-the-art machine learning algorithms struggle to learn the prope

'Their **intrinsic** nature makes them prone to **structural** modification, **which** further increases their **versatility** for a variety of applications. However, the search space for possible configurations is **vast**. Thus, the ability to predict the **properties** of such crystals efficiently **remains** a vital task. In this **study**, we focus on predicting the properties of **metallic** crystals blended with defects, substitutions, and **impurities**. State-of-the-art **machine** learning algorithms struggle to learn the properties of crystals’ defects accurately.'

### Next step MASKING p =0.3

In [420]:
p =0.3
response, job_id, data = fineTuningOpenai(training_data,
                 validation_data,
                 expert,
                 f"tmp_ai4material_v{p}",
                 p=p,
                 model="ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v0-2:9xuy5Jqv",  # gpt-3.5-turbo-1106
                 suffix=f"ai4materials_v{p}",)




Masked training data 83 validation data 21
Training file ID: file-EWhiHueGlalALXTBoP36NG1Z
Validation file ID: file-zXyseBW7zspmAy2sMgUAIZ5y
Job ID: ftjob-vR2VsXAwHzuLgRICRvG25gy3
Status: validating_files
Job ID: ftjob-vR2VsXAwHzuLgRICRvG25gy3
Status: validating_files
Trained Tokens: None


In [422]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()


for event in events:
    print(event.message)

Step 18/63: training loss=0.94, validation loss=0.39
Step 19/63: training loss=0.69, validation loss=0.66
Step 20/63: training loss=0.78, validation loss=0.26, full validation loss=0.46
Step 21/63: training loss=1.06, validation loss=0.91
Step 22/63: training loss=0.99, validation loss=0.75
Step 23/63: training loss=0.58, validation loss=0.59
Step 24/63: training loss=0.63, validation loss=0.64
Step 25/63: training loss=0.85, validation loss=0.20
Step 26/63: training loss=0.75, validation loss=0.72
Step 27/63: training loss=1.00, validation loss=1.02
Step 28/63: training loss=0.92, validation loss=0.42
Step 29/63: training loss=0.79, validation loss=0.63
Step 30/63: training loss=0.90, validation loss=0.34
Step 31/63: training loss=0.85, validation loss=0.62
Step 32/63: training loss=0.63, validation loss=0.95
Step 33/63: training loss=0.79, validation loss=0.59
Step 34/63: training loss=0.96, validation loss=0.57
Step 35/63: training loss=0.53, validation loss=0.44
Step 36/63: trainin

In [426]:
ft_model = events[-2].message.split(" ")[-1]

In [ ]:
OpenAIEmbedding.create(model="text-embedding-ada-002",

In [427]:
j=1
test_messages = []
test_messages.append({"role": "system", 
                      "content": data["validation"][j]['messages'][0]['content']})

test_messages.append({"role": "user", 
                      "content": data["validation"][j]['messages'][1]['content']})
print(test_messages[1]['content'])
print()
print(data["validation"][j]['messages'][2]['content'])

get_completion(test_messages, ft_model)
             #  "ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v0-3:9xv8GRrq")


Their two-dimensional nature  ...  them prone to  ...  modification, which further increases their tunability for a variety of  ...  However, the search space for possible  ...  is vast. Thus, the  ...  to predict the properties of  ...   ...   ...  becomes a vital  ...  In this paper, we focus on predicting the properties of such crystals blended  ...  defects, substitutions, and vacancies.  ...   ...   ...  algorithms  ...  to  ...  the properties of  ...  defects accurately.

Their two-dimensional nature makes them prone to chemical modification, which further increases their tunability for a variety of applications. However, the search space for possible configurations is vast. Thus, the ability to predict the properties of such crystals efficiently becomes a vital task. In this paper, we focus on predicting the properties of such crystals blended with defects, substitutions, and vacancies. State-of-the-art machine learning algorithms struggle to learn the properties of crystals’ d

'Their two-dimensional nature **makes** them prone to **structural** modification, which further increases their tunability for a variety of **applications**. However, the search space for possible **configurations** is vast. Thus, the **ability** to predict the properties of **two-dimensional materials** becomes a vital **task**. In this paper, we focus on predicting the properties of such crystals blended **with** defects, substitutions, and vacancies. **Machine learning** algorithms **are employed** to **model** the properties of **these** defects accurately.'

### Next step MASKING p =0.4

In [428]:
p =0.4
print("Fine-Tune:",ft_model)
response, job_id, data = fineTuningOpenai(training_data,
                 validation_data,
                 expert,
                 f"tmp_ai4material_v{p}",
                 p=p,
                 model=ft_model,  # gpt-3.5-turbo-1106
                 suffix=f"ai4materials_v{p}",)




Masked training data 83 validation data 21
Training file ID: file-pE1F6bEcHTFiCjOJeejpSFpu
Validation file ID: file-oTspWNLJ9KYJuupu56BsGal6
Job ID: ftjob-v1HwcNteEegHKoU0pylbtEOD
Status: validating_files
Job ID: ftjob-v1HwcNteEegHKoU0pylbtEOD
Status: validating_files
Trained Tokens: None


In [433]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()


for event in events:
    print(event.message)

Step 18/63: training loss=1.05, validation loss=0.67
Step 19/63: training loss=1.39, validation loss=0.97
Step 20/63: training loss=1.42, validation loss=0.50, full validation loss=0.66
Step 21/63: training loss=1.14, validation loss=1.12
Step 22/63: training loss=1.27, validation loss=1.08
Step 23/63: training loss=0.95, validation loss=0.68
Step 24/63: training loss=0.99, validation loss=1.10
Step 25/63: training loss=1.13, validation loss=0.40
Step 26/63: training loss=1.18, validation loss=0.86
Step 27/63: training loss=1.03, validation loss=1.55
Step 28/63: training loss=0.86, validation loss=0.70
Step 29/63: training loss=1.33, validation loss=1.00
Step 30/63: training loss=1.10, validation loss=0.48
Step 31/63: training loss=1.15, validation loss=0.72
Step 32/63: training loss=1.13, validation loss=1.48
Step 33/63: training loss=1.16, validation loss=0.82
Step 34/63: training loss=0.88, validation loss=0.62
Step 35/63: training loss=0.98, validation loss=0.57
Step 36/63: trainin

In [434]:
ft_model = events[-2].message.split(" ")[-1]
ft_model

'ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v0-4:9xvNMZT0'

In [435]:
j=1
test_messages = []
test_messages.append({"role": "system", 
                      "content": data["validation"][j]['messages'][0]['content']})

test_messages.append({"role": "user", 
                      "content": data["validation"][j]['messages'][1]['content']})
print(test_messages[1]['content'])
print()
print(data["validation"][j]['messages'][2]['content'])

get_completion(test_messages, ft_model)
             #  "ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v0-3:9xv8GRrq")


Their two-dimensional nature  ...   ...   ...  to  ...   ...   ...  further increases their tunability for a variety of applications.  ...  the search  ...  for  ...  configurations is vast. Thus, the ability to  ...  the properties of such  ...   ...  becomes a  ...  task. In this paper, we focus on  ...  the  ...  of such  ...  blended  ...  defects, substitutions, and  ...  State-of-the-art machine  ...  algorithms struggle to learn the properties of crystals’ defects accurately.

Their two-dimensional nature makes them prone to chemical modification, which further increases their tunability for a variety of applications. However, the search space for possible configurations is vast. Thus, the ability to predict the properties of such crystals efficiently becomes a vital task. In this paper, we focus on predicting the properties of such crystals blended with defects, substitutions, and vacancies. State-of-the-art machine learning algorithms struggle to learn the properties of crysta

'Their two-dimensional nature **contributes significantly** to **enhanced performance**, which further increases their tunability for a variety of applications. **Consequently**, the search **space** for **optimal** configurations is vast. Thus, the ability to **manipulate** the properties of such **materials** becomes a **challenging** task. In this paper, we focus on **investigating** the **effects** of such **materials**, blended **with** defects, substitutions, and **interstitials**. State-of-the-art machine **learning** algorithms struggle to learn the properties of crystals’ defects accurately.'

### Next step MASKING p =0.5

In [436]:
p =0.5
print("Fine-Tune:",ft_model)
response, job_id, data = fineTuningOpenai(training_data,
                 validation_data,
                 expert,
                 f"tmp_ai4material_v{p}",
                 p=p,
                 model=ft_model,  # gpt-3.5-turbo-1106
                 suffix=f"ai4materials_v{p}",)




Fine-Tune: ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v0-4:9xvNMZT0
Masked training data 83 validation data 21
Training file ID: file-PMLMu5V5v7ZXri3pyA54nya8
Validation file ID: file-CwLw8gY009ioAOLnYdqtfmDI
Job ID: ftjob-5eO1UihoRUsKZNzQ1yqKggOB
Status: validating_files
Job ID: ftjob-5eO1UihoRUsKZNzQ1yqKggOB
Status: validating_files
Trained Tokens: None


In [444]:
response = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

events = response.data
events.reverse()


for event in events:
    print(event.message)

Step 18/63: training loss=1.32, validation loss=0.88
Step 19/63: training loss=1.42, validation loss=1.17
Step 20/63: training loss=1.46, validation loss=0.37, full validation loss=0.68
Step 21/63: training loss=1.19, validation loss=1.01
Step 22/63: training loss=1.40, validation loss=1.03
Step 23/63: training loss=1.34, validation loss=0.97
Step 24/63: training loss=1.21, validation loss=1.11
Step 25/63: training loss=1.40, validation loss=0.48
Step 26/63: training loss=1.44, validation loss=0.96
Step 27/63: training loss=1.35, validation loss=1.16
Step 28/63: training loss=1.24, validation loss=1.00
Step 29/63: training loss=1.16, validation loss=0.85
Step 30/63: training loss=1.63, validation loss=0.71
Step 31/63: training loss=1.53, validation loss=0.69
Step 32/63: training loss=1.54, validation loss=1.19
Step 33/63: training loss=1.57, validation loss=0.74
Step 34/63: training loss=1.38, validation loss=0.88
Step 35/63: training loss=1.56, validation loss=0.88
Step 36/63: trainin

In [446]:
_l = events[-2].message.split(" ")
ft_model = _l[-1] if _l[-2] == "created:" else None
ft_model

'ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v0-5:9xvYboC9'

In [447]:
j=1
test_messages = []
test_messages.append({"role": "system", 
                      "content": data["validation"][j]['messages'][0]['content']})

test_messages.append({"role": "user", 
                      "content": data["validation"][j]['messages'][1]['content']})
print(test_messages[1]['content'])
print()
print(data["validation"][j]['messages'][2]['content'])

get_completion(test_messages, ft_model)
             #  "ft:gpt-4o-mini-2024-07-18:constructor-tech:ai4materials-v0-3:9xv8GRrq")


Their  ...  nature makes  ...  prone to  ...   ...  which further  ...   ...   ...  for a  ...  of applications. However, the  ...  space for  ...  configurations is  ...   ...  the ability to  ...  the properties of  ...  crystals  ...  becomes a vital  ...  In this paper, we focus on predicting the  ...  of such  ...  blended with defects,  ...  and vacancies.  ...   ...  learning  ...  struggle to  ...  the properties of  ...  defects accurately.

Their two-dimensional nature makes them prone to chemical modification, which further increases their tunability for a variety of applications. However, the search space for possible configurations is vast. Thus, the ability to predict the properties of such crystals efficiently becomes a vital task. In this paper, we focus on predicting the properties of such crystals blended with defects, substitutions, and vacancies. State-of-the-art machine learning algorithms struggle to learn the properties of crystals’ defects accurately.


'Their **intrinsic** nature makes **materials** prone to **defects** **and** which further **complicates** **the** **design** for a **wide range** of applications. However, the **available** space for **configuring** configurations is **limited**, **thus** the ability to **predict** the properties of **defective** crystals **accurately** becomes a vital **challenge**. In this paper, we focus on predicting the **properties** of such **materials** blended with defects, **dislocations**, and vacancies. **Traditional** **machine** learning **models** struggle to **capture** the properties of **complex** defects accurately.'